In [21]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle as pkl

In [22]:
file = open('/kaggle/input/main-twitter/vocabulary.pkl', "rb")
vocabulary = pkl.load(file)
file.close()

In [23]:
df = pd.read_csv('/kaggle/input/main-twitter/main.csv')

In [24]:
df = df[df['class'] != '-']
df.drop(['name'], axis=1, inplace=True)
df.dropna(how='any', inplace=True)
df.reset_index(drop=True, inplace=True)
df['class'] = df['class'].astype(int)
df = df[df['class'] >= 0]

In [25]:
def isInVocabulary(word, voc=vocabulary):
    """
    Searchs for word from dictonary based on first letter
    """
    first_letter = word[0]
    if first_letter not in voc:
        return False
    if word in voc[first_letter]:
        return True
    else:
        return False

In [26]:
def stem(string):
    l=[]
    vowels=["a","ı","o","u","e","ə","i"]
    string=string.split()
    for i in string:
        if i.isupper() or (string.index(i)!=0 and i[0].isupper()):
            # xüsusi isimlər və abbr. üçün
            l.append(i)
        else:
            for j in range(len(i),0,-1):
                if isInVocabulary(i[:j].casefold()): # i[:j].casefold() in words:
                    # adi şəkilçilər üçün
                    l.append(i[:j])
                    break
                elif i[j-1] in vowels and (i[j-2]=="y" or i[j-2]=="ğ") :
                    # bitişdirici samitlər üçün
                    if isInVocabulary((i[:j-2]+"k").casefold()): # (i[:j-2]+"k").casefold() in words:
                        l.append(i[:j-2]+"k")
                        break
                    elif isInVocabulary((i[:j-2]+"q").casefold()): # (i[:j-2]+"q").casefold() in words:
                        l.append(i[:j-2]+"q")
                        break
    return ' '.join(l)

In [27]:
df['Text'] = df['Text'].apply(lambda row: stem(row))

In [28]:
stemmed_data = df.copy()
stemmed_data.to_csv('clean_stemmed_data.csv', index=False)

In [29]:
!pip install lazy-text-predict

In [30]:
from warnings import filterwarnings
filterwarnings('ignore')

In [31]:
from lazytextpredict import basic_classification

trial=basic_classification.LTP(Xdata=list(df["Text"].values),Ydata=list(df["class"].values), x_col='Text', y_col='class', models='all') 
# Xdata is a list of text entries, and Ydata is a list of corresponding labels.
# csv and xlsx give options to load data from those file formats (you can pass the file or the file's location)
# x_col and y_col are strings that specify the columns of the # text and label columns in your csv or xlsx file respectively.
# You can choose between 'transformers'-based, 'count-vectorizer'-based, and 'all' models.

trial.run(training_epochs=5) 
#This trains the models specified above on the data you loaded. 
#Here you can specify the number of training epochs. 
#Fewer training epochs will give poorer performance, but will run quicker to allow debugging.

trial.print_metrics_table()

X_train length: 5760
X_test length: 641
Y_train length: 5760
Y_test length: 641
Training on a dataset with 3 labels


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 5760
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1800
  Number of trainable parameters = 109484547
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


...

{'eval_loss': 0.7989315390586853, 'eval_accuracy': 0.7020280811232449, 'eval_f1': 0.7003852680695669, 'eval_precision': 0.7019295922980677, 'eval_recall': 0.7020280811232449, 'eval_full_report': {'0': {'precision': 0.5652173913043478, 'recall': 0.5855855855855856, 'f1-score': 0.575221238938053, 'support': 111}, '1': {'precision': 0.7125748502994012, 'recall': 0.6134020618556701, 'f1-score': 0.6592797783933518, 'support': 194}, '2': {'precision': 0.7409470752089137, 'recall': 0.7916666666666666, 'f1-score': 0.7654676258992805, 'support': 336}, 'accuracy': 0.7020280811232449, 'macro avg': {'precision': 0.6729131056042208, 'recall': 0.6635514380359742, 'f1-score': 0.6666562144102285, 'support': 641}, 'weighted avg': {'precision': 0.7019295922980677, 'recall': 0.7020280811232449, 'f1-score': 0.7003852680695669, 'support': 641}}, 'eval_runtime': 10.9735, 'eval_samples_per_second': 58.414, 'eval_steps_per_second': 1.002, 'epoch': 5.0}


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Training on a dataset with 3 labels
ERROR
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Training on a dataset with 3 labels
ERROR
best parameters are:
{'clf__alpha': 0.1, 'clf__fit_prior': True, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 2)}
{'eval_loss': 0.24648985959438377, 'eval_accuracy': 0.7535101404056163, 'eval_f1': 0.7198045374146003, 'eval_precision': 0.7703727928854219, 'eval_recall': 0.6927701818165737, 'eval_full_report': '              precision    recall  f1-score   support\n\n           0       0.77      0.54      0.63       111\n           1       0.82      0.66      0.73       194\n           2       0.73      0.88      0.80       336\n\n    accuracy                           0.75       641\n   macro avg       0.77      0.69      0.72       641\nweighted avg       0.76      0.75      0.75       641\n'}
                    Model            loss        accuracy              f1       precision          recall
        bert-base-uncased         0.85345         0.72699         0.72214         0.72753         0.72699
           albert-base-v2         0.96